In [1]:
import pandas as pd
df = pd.read_csv("Sales.csv")

In [2]:
import pandas as pd

unique_pizza_ids = df['pizza_name_id'].unique()

dataframes = {}

for pizza_id in unique_pizza_ids:
    df_filtered = df[df['pizza_name_id'] == pizza_id][['week_number', 'quantity', 'order_date']]
    dataframes[pizza_id] = df_filtered

In [3]:
unique_pizza_ids[0]

'hawaiian_m'

In [4]:
import pandas as pd
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_percentage_error

results = {}

for j in range(0,91):

    example_pizza_id = unique_pizza_ids[j]
    df = dataframes[example_pizza_id]

    df_weekly = df.groupby('week_number').sum().reset_index()
    df_weekly.rename(columns={'week_number': 'ds', 'quantity': 'y'}, inplace=True)

    split_point = int(len(df_weekly) * 0.7)
    train_df = df_weekly[:split_point]
    test_df = df_weekly[split_point:]

    sarima_model = SARIMAX(train_df['y'], order=(1,1,1), seasonal_order=(1,1,1,52))  # You can tune the orders
    sarima_fit = sarima_model.fit(disp=False)

    sarima_forecast = sarima_fit.forecast(steps=len(test_df))
    sarima_forecast = pd.DataFrame(sarima_forecast, index=test_df['ds'], columns=['yhat'])

    mape = mean_absolute_percentage_error(test_df['y'], sarima_forecast['yhat'])
    week_forecast = sarima_forecast.iloc[-1]

    results[f'unique_pizza_ids[{j}]'] = {
        'Pizza_id': unique_pizza_ids[j],
        'MAPE': mape,
        'Next_Week_Prediction': week_forecast
    }


c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
c:\Users\ANNIE CHRISTINA G\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal ARMA. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'


ValueError: Input contains NaN.